# Production Modeling
- Recommender Model - Content Based Filtering
    - Recommend items that are similar to those that a user liked in the past
- Providing users with top matches based on the matching criteria
- User can choose to see additional profiles based on ones they liked from the initial matches


**-->** This notebook contains different analyses of combinations of user input values based on their orientation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics.pairwise import cosine_similarity
from category_encoders import OneHotEncoder

In [74]:
# read in full cupid data
cupid = pd.read_pickle('data/clean_cupid.pkl')
cupid.drop(columns = ['status', 'location'], inplace = True)

In [3]:
cupid.head(3)

,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
0,22,m,straight,a little extra,strictly anything,socially,never,"doesn't have kids, but might want them",likes dogs and likes cats,agnosticism and very serious about it,sometimes
1,35,m,straight,average,mostly other,often,sometimes,"doesn't have kids, but might want them",likes dogs and likes cats,agnosticism but not too serious about it,no
2,38,m,straight,thin,anything,socially,never,doesn't have kids,has cats,atheism,no


In [75]:
# read in grouped cupid data
cupid_df = pd.read_pickle('data/grouped_cupid.pkl')

In [5]:
cupid_df.head(3)

,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
0,22,m,straight,average,anything,sometimes,no,"doesn't have kid(s), but wants kid(s)",likes dogs and cats,agnosticism,sometimes
1,35,m,straight,average,other,yes,sometimes,"doesn't have kid(s), but wants kid(s)",likes dogs and cats,doesn't matter,no
2,38,m,straight,thin,anything,sometimes,no,doesn't have kids,likes cats,doesn't matter,no


---

## Recommender Modeling - Content-Based Filtering

**Matching Criteria:**
- **Orientation**: Most important to match a user based on their sex and orientation
    - For example, Straight Male matched with either a Straight Female or a Bisexual Female
- **Age Range**: Everyone is in different stages of their life, depending on their age or where they are in life
    - For example, perhaps someone younger is looking for more mature individuals
- **Religion**: Portrays one's personal and cultural values. 
    - If religion matters to a user, the User would be recommended profiles of the same religion
    - If religion doesn’t matter, the User would be recommended profiles of those who are atheists / not serious about religion
- **Offspring Sentiment**: To take a relationship to the next level, the topic of children normally come up
    - If a user *wanted kid(s)*, they'd be recommended profiles of users who have kid(s)
    - If a user *didn't want kid(s)*, they'd be recommended profiles of users who didn't have kid(s)

In [315]:
# separates data by orientation

def orientation(orientation, sex, religion, lowest_age, highest_age):
    """
    This function separates takes in a user's orientation and sex, and filters the data based on the matching criteria.
    
    invalue (array): survey/streamlit app responses
    religion: religion of user
    lowest_age / highest_age: age range preference of partner recommendation
    """
    
    # straight female looking for straight/bi male
    if orientation == 'straight/bi' and sex == 'm':
        return cupid_df[(((cupid_df['sex'] == 'm') & (cupid_df['orientation'] == 'straight')) | ((cupid_df['sex'] == 'm') & (cupid_df['orientation'] == 'bisexual'))) \
                    & (cupid_df['religion'] == religion) & (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)]
    
    # straight male looking for straight/bi female
    elif orientation == 'straight/bi' and sex == 'f':
        return cupid_df[(((cupid_df['sex'] == 'f') & (cupid_df['orientation'] == 'straight')) | ((cupid_df['sex'] == 'f') & (cupid_df['orientation'] == 'bisexual'))) \
                & (cupid_df['religion'] == religion) & (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)]
    
    # gay male looking for gay/bi male
    elif orientation == 'gay/bi' and sex == 'm':
        return cupid_df[(cupid_df['sex'] == 'm') & ((cupid_df['orientation'] == 'gay') | (cupid_df['orientation'] == 'bisexual')) & \
             (cupid_df['religion'] == religion) & (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)]
    
    # gay female looking for gay/bi female
    elif orientation == 'gay/bi' and sex == 'f':
        return cupid_df[(cupid_df['sex'] == 'f') & ((cupid_df['orientation'] == 'gay') | (cupid_df['orientation'] == 'bisexual')) & \
             (cupid_df['religion'] == religion) & (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)]
    
    # bi female looking for bi individual or straight male or gay female
    elif orientation == 'bisexual' and sex == 'f':
        return cupid_df[((cupid_df['sex'] == 'f') & (cupid_df['orientation'] == 'gay') & (cupid_df['religion'] == religion) & \
                         (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)) | ((cupid_df['sex'] == 'm') & (cupid_df['orientation'] == 'straight') & \
                        (cupid_df['religion'] == religion) & (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)) | \
                     (cupid_df['orientation'] == 'bisexual')  & (cupid_df['religion'] == religion) & (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)]
    
    # bi male looking for bi individual or straight female or gay male
    elif orientation == 'bisexual' and sex == 'm':
        return cupid_df[((cupid_df['sex'] == 'm') & (cupid_df['orientation'] == 'gay') & (cupid_df['religion'] == religion) & \
                         (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)) | ((cupid_df['sex'] == 'f') & (cupid_df['orientation'] == 'straight') & \
                        (cupid_df['religion'] == religion) & (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)) | \
                     (cupid_df['orientation'] == 'bisexual')  & (cupid_df['religion'] == religion) & (cupid_df['age'] >= lowest_age) & (cupid_df['age'] <= highest_age)]

In [68]:
# performs recommendation

def lover_recommender_test8(invalue, religion, lowest_age, highest_age):
    """
    This function takes in a user's inputs and returns the top matches based on the matching criteria and highest cosine similarities.
    
    invalue (array): survey/streamlit app responses
    religion: religion of user
    lowest_age / highest_age: age range preference of partner recommendation
    """
   
    # convert input from array to DataFrame
    invalue_df = pd.DataFrame(invalue).T.rename(columns = {i:j for i,j in zip(np.arange(11), cupid_df.columns)})

    # ----------------
    
    # straight female looking for straight/bi male
    if invalue_df['orientation'].unique()[0] == 'straight' and invalue_df['sex'].unique()[0] == 'f':
        
        # split data by orientation
        orientation_df = orientation('straight/bi', 'm', religion, lowest_age, highest_age)
        
        # call 'invalue_to_similarity' function to return top 5 similarities
        try: 
            return offspring_subset(invalue_df, orientation_df)
        except ValueError:
            return "Apologies, we don't have any profiles in the database that relate to your age range, orientation, and religious inputs at the moment."
    
    # straight male looking for straight/bi female
    elif invalue_df['orientation'].unique()[0] == 'straight' and invalue_df['sex'].unique()[0] == 'm':
        
        # split data by orientation
        orientation_df = orientation('straight/bi', 'f', religion, lowest_age, highest_age)

        # call 'invalue_to_similarity' function to return similarities
        try: 
            return offspring_subset(invalue_df, orientation_df)
        except ValueError:
            return "Apologies, we don't have any profiles in the database that relate to your age range, orientation, and religious inputs at the moment."
        
    # gay male looking for gay/bi male
    elif invalue_df['orientation'].unique()[0] == 'gay' and invalue_df['sex'].unique()[0] == 'm':
        
        # split data by orientation
        orientation_df = orientation('gay/bi', 'm', religion, lowest_age, highest_age)
        
        # call 'invalue_to_similarity' function to return similarities
        try: 
            return offspring_subset(invalue_df, orientation_df)
        except ValueError:
            return "Apologies, we don't have any profiles in the database that relate to your age range, orientation, and religious inputs at the moment."
    
    # gay female looking for gay/bi female
    elif invalue_df['orientation'].unique()[0] == 'gay' and invalue_df['sex'].unique()[0] == 'f':
        
        # split data by orientation
        orientation_df = orientation('gay/bi', 'f', religion, lowest_age, highest_age)
        
        # call 'invalue_to_similarity' function to return similarities
        try: 
            return offspring_subset(invalue_df, orientation_df)
        except ValueError:
            return "Apologies, we don't have any profiles in the database that relate to your age range, orientation, and religious inputs at the moment."
    
    # bi female looking for bi individual or gay female or straight male
    elif (invalue_df['orientation'].unique()[0] == 'bisexual' and invalue_df['sex'].unique()[0] == 'f'):
        
        # split data by orientation
        orientation_df = orientation('bisexual', 'f', religion, lowest_age, highest_age)
        
        # call 'invalue_to_similarity' function to return similarities
        try: 
            return offspring_subset(invalue_df, orientation_df)
        except ValueError:
            return "Apologies, we don't have any profiles in the database that relate to your age range, orientation, and religious inputs at the moment."    
    
    # bi male looking for bi individual or gay male or straight female
    elif (invalue_df['orientation'].unique()[0] == 'bisexual' and invalue_df['sex'].unique()[0] == 'm'):
        
    # split data by orientation
        orientation_df = orientation('bisexual', 'm', religion, lowest_age, highest_age)

        # call 'invalue_to_similarity' function to return similarities
        try: 
            return offspring_subset(invalue_df, orientation_df)
        except ValueError:
            return "Apologies, we don't have any profiles in the database that relate to your age range, orientation, and religious inputs at the moment."

In [69]:
# offspring

def offspring_subset(invalue_df, orientation_df):
    """
    This function further filters the orientation data based on offspring sentiment. 
    
    invalue_df: converted DataFrame of user inputs
    orientation_df: DataFrame of all people of that orientation
    """
    
    # split by offspring preference
    if "doesn't have kid(s), but wants kid(s)" in invalue_df['offspring'].unique():
        orientation_df = orientation_df[(orientation_df['offspring'] == "has kid(s)") | (orientation_df['offspring'] == "has kid(s) and wants more") | \
                                        (orientation_df['offspring'] == "has kid(s), but doesn't want more")]
    elif "doesn't have kids, and doesn't want any" in invalue_df['offspring'].unique():
        orientation_df = orientation_df[(orientation_df['offspring'] == "doesn't have kids") | (orientation_df['offspring'] == "doesn't want kids") | \
                                        (orientation_df['offspring'] == "doesn't have kids, and doesn't want any")]
    elif "has kid(s), but doesn't want more" in invalue_df['offspring'].unique():
        orientation_df = orientation_df[(orientation_df['offspring'] == "doesn't have kids") | (orientation_df['offspring'] == "doesn't have kid(s), but wants kid(s)") | \
                                        (orientation_df['offspring'] == "wants kid(s)")]
    elif "has kid(s) and wants more" in invalue_df['offspring'].unique():
        orientation_df = orientation_df[(orientation_df['offspring'] == "has kid(s)") | (orientation_df['offspring'] == "has kid(s) and wants more") | \
                                        (orientation_df['offspring'] == "wants kid(s)")]
    
    # cosine similarities
    return invalue_to_similarity(invalue_df, orientation_df)

In [113]:
# function to ohe return the cosine similarity based on orientation

def invalue_to_similarity(invalue_df, orientation_df):
    """
    This function encodes the user input (invalue_df) and filtered data (based on matching criteria -- orientation, religion, age range, offspring sentiment) into 0/1
    and returns the top matches based on highest cosine similarity. 
    
    Note: Values returned in the top matches are the original database values, not the grouped / generalized values.
    
    invalue_df: converted DataFrame of user inputs
    orientation_df: DataFrame of all people of that orientation
    """
    
    # concat input values to orientation df to prep for cosine similarity
    df = pd.concat([orientation_df, invalue_df])

    # ohe
    df_encoded = OneHotEncoder(use_cat_names = True).fit_transform(df)
    
    # make cosine_similarity input (input X)
    cosine_input = pd.DataFrame(df_encoded.iloc[-(len(invalue_df))]).T
    
    # drop last encoded row (input Y -- data for input X to reference)
    df_encoded.drop(df_encoded.tail(len(invalue_df)).index, inplace = True)
    
    # cosine_similarity(X, y)
    similarity = cosine_similarity(cosine_input, df_encoded)
    
    # return top 5 matches
    top5 = pd.DataFrame(similarity.tolist()[0], columns = ['similarity'], index = df_encoded.index).sort_values(by = 'similarity', ascending = False).iloc[:5]
    
    # return top 5 matches in a df with cosine similarities
    results = pd.DataFrame(columns = cupid.columns)

    for i in top5.index:
        results = results.append(pd.DataFrame(cupid.loc[i]).T)

    matches = pd.merge(top5, results, on = top5.index)
    matches.rename(columns = {'key_0' : 'user_id'}, inplace = True)
    matches.set_index('user_id', drop = True, inplace = True)
    
    return matches

In [282]:
# function for the user to tell which matches they like and to return additional matches, if the user chooses

def more_matches2(invalue, profile_lis, religion, lowest_age, highest_age):
    """
    This function returns additional matches based on profile(s) a user liked from the initial recommendations.
    
    invalue(array): user inputs
    profile_lis (list): list of indexes of the profiles the user liked from the initial recommendations
    religion: religion of user
    lowest_age / highest_age: age range preference of partner recommendation
    """
    
    invalue_df = pd.DataFrame(invalue).T.rename(columns = {i:j for i,j in zip(np.arange(11), cupid_df.columns)})
    likes_df = pd.DataFrame(columns = cupid_df.columns)
        
    # collect liked profiles
    for person in profile_lis:
        likes_df = likes_df.append(cupid_df.loc[person])
        
    answer = str(input('Want to see more profiles? (yes/no)'))
    
    if answer == 'yes':

        # straight female
        if invalue_df['sex'].unique()[0] == 'f' and invalue_df['orientation'].unique()[0] == 'straight':

            # split data by orientation
            orientation_df = orientation('straight/bi', 'm', religion, lowest_age, highest_age)

            print("Here are more profiles, based on the profile(s) of interest from earlier:")
            return offspring_subset(likes_df, orientation_df)

        # straight male
        elif invalue_df['sex'].unique()[0] == 'm' and invalue_df['orientation'].unique()[0] == 'straight':

            # split data by orientation
            orientation_df = orientation('straight/bi', 'f', religion, lowest_age, highest_age)

            print("Here are more profiles, based on the profile(s) of interest from earlier:")
            return offspring_subset(likes_df, orientation_df)

        # gay female
        elif invalue_df['sex'].unique()[0] == 'f' and invalue_df['orientation'].unique()[0] == 'gay':

            # split data by orientation
            orientation_df = orientation('gay/bi', 'f', religion, lowest_age, highest_age)

            print("Here are more profiles, based on the profile(s) of interest from earlier:")
            return offspring_subset(likes_df, orientation_df)

        # gay male
        elif invalue_df['sex'].unique()[0] == 'm' and invalue_df['orientation'].unique()[0] == 'gay':

            # split data by orientation
            orientation_df = orientation('gay/bi', 'm', religion, lowest_age, highest_age)

            print("Here are more profiles, based on the profile(s) of interest from earlier:")
            return offspring_subset(likes_df, orientation_df)

        # bi female looking for bi individual or gay female or straight male
        elif (invalue_df['orientation'].unique()[0] == 'bisexual' and invalue_df['sex'].unique()[0] == 'f'):

            # split data by orientation
            orientation_df = orientation('bisexual', 'f', religion, lowest_age, highest_age)

            print("Here are more profiles, based on the profile(s) of interest from earlier:")
            return offspring_subset(likes_df, orientation_df)

        # bi male looking for bi individual or gay male or straight female
        elif (invalue_df['orientation'].unique()[0] == 'bisexual' and invalue_df['sex'].unique()[0] == 'm'):

            # split data by orientation
            orientation_df = orientation('bisexual', 'm', religion, lowest_age, highest_age)

            print("Here are more profiles, based on the profile(s) of interest from earlier:")
            return offspring_subset(likes_df, orientation_df)
            
    else:
        print('Here are the profiles you liked:')
        return(likes_df)

In [313]:
# user inputs in question form -- similar to layout of Streamlit App

# age + age range
age = int(input('How old are you?'))
print("What's your age range?")
lowest_age = int(input('Lowest age?'))
highest_age = int(input('Highest age?'))

# dropdowns
sex = str(input('What gender do you identify as?'))
orientation = str(input('What sexual orientation do you identify with?'))
body_type = str(input("What's your body type?"))
diet = str(input('What does your diet consist of?'))
drinks = str(input('Do you consume alcoholic beverages?'))
drugs = str(input('Do you use drugs?'))

# offspring
# do you have any children? yes/no
any_children = str(input('Do you have any children?'))

# do you want any [more] children? yes/no
more_children = str(input('Do you want any [more] children?'))

if any_children == 'no' and more_children == 'yes':
    offspring = "doesn't have kid(s), but wants kid(s)"
elif any_children == 'no' and more_children == 'no':
    offspring = "doesn't have kids, and doesn't want any"
elif any_children == 'yes' and more_children == 'no':
    offspring = "has kid(s), but doesn't want more"
elif any_children == 'yes' and more_children == 'yes':
    offspring = "has kid(s) and wants more"

pets = str(input("What's your sentiment on dogs and/or cats"))
smokes = str(input("Do you smoke?"))

# religion
religion_matter = str(input('Does religion matter?'))
if religion_matter == 'yes':
    religion = str(input("What's your religion?"))
else:
    religion = "doesn't matter"

invalue = np.array([age, sex, orientation, body_type, diet, drinks, drugs, offspring, pets, religion, smokes])

How old are you? 45


What's your age range?


Lowest age? 35
Highest age? 48
What gender do you identify as? m
What sexual orientation do you identify with? stright
What's your body type? fit
What does your diet consist of? anything
Do you consume alcoholic beverages? yes
Do you use drugs? no
Do you have any children? yes
Do you want any [more] children? yes
What's your sentiment on dogs and/or cats likes dogs
Do you smoke? no
Does religion matter? yes
What's your religion? christianity


---

## Analyses of Different User Inputs

In [273]:
# straight male

invalue13 = ['45', 'm', 'straight', 'fit', 'anything', 'yes', 'no', "has kid(s) and wants more", 'likes dogs', 'christianity', 'no']
highest_age13 = 48
lowest_age13 = 35
religion13 = 'christianity'

lover_recommender_test8(invalue13, religion13, lowest_age13, highest_age13)

C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,similarity,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
user_id,,,,,,,,,,,,
12980,0.636364,45,f,straight,fit,anything,socially,never,"has a kid, and might want more",dislikes dogs and dislikes cats,christianity,no
54734,0.636364,41,f,straight,fit,anything,socially,never,"has a kid, and might want more",likes dogs and has cats,christianity,no
29976,0.636364,42,f,straight,thin,anything,often,never,has a kid,has dogs,christianity,no
50152,0.636364,37,f,straight,curvy,anything,often,never,has kids,likes dogs,christianity,no
23127,0.636364,37,f,straight,rather not say,anything,not at all,never,"has a kid, and might want more",likes dogs,christianity and somewhat serious about it,no


In [274]:
# returning additional profiles based on any of the ones the user liked

answer = str(input('Any profile(s) catch your eye? (yes/no)'))

profile_lis = []
while answer == 'yes':
    profile = int(input('Name one profile (user_id) of interest: '))
    profile_lis.append(profile)
    answer = str(input('Any other profile(s) caught your eye? (yes/no)'))
    
more_matches2(invalue13, profile_lis, religion13, lowest_age13, highest_age13)

Any profile(s) catch your eye? (yes/no) yes
Name one profile (user_id) of interest:  54734
Any other profile(s) caught your eye? (yes/no) yes
Name one profile (user_id) of interest:  50152
Any other profile(s) caught your eye? (yes/no) no
Want to see more profiles? (yes/no) yes


Here are more profiles, based on the profile(s) of interest from earlier:


C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,similarity,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
user_id,,,,,,,,,,,,
30913,0.909091,41,f,straight,average,mostly anything,socially,never,"has kids, and might want more",likes dogs and likes cats,christianity and very serious about it,no
223,0.818182,42,f,straight,athletic,anything,socially,never,has kids,has dogs and has cats,christianity and somewhat serious about it,no
12980,0.818182,45,f,straight,fit,anything,socially,never,"has a kid, and might want more",dislikes dogs and dislikes cats,christianity,no
55432,0.818182,35,f,straight,athletic,mostly anything,socially,never,has a kid,likes dogs and likes cats,christianity and somewhat serious about it,no
4713,0.818182,41,f,straight,a little extra,mostly anything,socially,never,"has kids, and might want more",likes dogs,christianity and very serious about it,no


#### **--> Analysis: Straight Male**

**Initial Matches based on matching criteria + cosine similarity**
- Based on the matching criteria, the user input is that of a **straight male** with an **age range between 35 - 48**, who **has kid(s) and wants more**, and is **Christian**. A user specifying their religion indicates that religion matters to them, so the recommendations return other users who are Christian / serious about Christianity as well. The initial recommendations returned straight females who all have kids.
- In terms of the other features that weren't part of the matching criteria, the user input has a diet consisting of `anything`, `yes` to drinks, and `no` to drugs, all returning recommended profiles similar for the most part, aside from user `23127` who indicated that they don't drink.

**Liked Profiles + Additional Matches**
- The User indicated that he liked profile `54734`, since the straight female is of a `fit` body type and likes dogs and has cats, whereas the User is also of a `fit` body type and likes dogs. Additionally, the User liked profile `50152`, to provide potential variation to the additional matches, since the straight female is of a `curvy` body type. Similarly, the straight female profile has kids and likes dogs, similar to that of the User's preferences.
- In terms of the **additional recommended profiles**, they are based off the liked profiles the User specified, while still performing cosine similarity against a filtered dataset based on the User's age range, orientation, and religion preference. The only matching criteria that is *directly related* to liked profile is `offspring sentiment`. Since one of the liked profiles indicate that the profile `has a kid, and might want more`, the additional matches returned have users that have kids as well.
- Additionally, since profile `54734` has a body type of `fit`, we're getting back several additional matches of useres with that of body type `athletic` or `fit.

In [262]:
# straight female

invalue3 = ['37', 'f', 'straight', 'full figured', 'anything', 'yes', 'no', "doesn't have kid(s), but wants kid(s)", 'likes dogs and cats', "doesn't matter", 'no']
highest_age3 = 40
lowest_age3 = 32
religion3 = "doesn't matter"

lover_recommender_test8(invalue3, religion3, lowest_age3, highest_age3)

C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,similarity,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
user_id,,,,,,,,,,,,
7206,0.636364,34,m,straight,fit,strictly anything,often,never,has a kid,likes dogs and likes cats,catholicism and laughing about it,no
3497,0.636364,35,m,straight,curvy,mostly anything,not at all,never,"has a kid, and might want more",likes dogs and likes cats,atheism,no
15047,0.636364,39,m,straight,fit,mostly anything,often,never,"has a kid, and wants more",likes dogs and likes cats,atheism and somewhat serious about it,no
54828,0.636364,39,m,straight,average,anything,very often,never,"has kids, but doesn't want more",likes dogs and likes cats,atheism,no
56272,0.636364,38,m,straight,fit,mostly anything,often,never,has kids,likes dogs and has cats,atheism,no


In [312]:
# returning additional profiles based on any of the ones the user liked

answer = str(input('Any profile(s) catch your eye? (yes/no)'))

profile_lis = []
while answer == 'yes':
    profile = int(input('Name one profile (user_id) of interest: '))
    profile_lis.append(profile)
    answer = str(input('Any other profile(s) caught your eye? (yes/no)'))
    
more_matches2(invalue3, profile_lis, religion3, lowest_age3, highest_age3)

Any profile(s) catch your eye? (yes/no) yes
Name one profile (user_id) of interest:  7206
Any other profile(s) caught your eye? (yes/no) yes
Name one profile (user_id) of interest:  15047
Any other profile(s) caught your eye? (yes/no) no
Want to see more profiles? (yes/no) yes


Here are more profiles, based on the profile(s) of interest from earlier:


C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,similarity,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
user_id,,,,,,,,,,,,
42530,0.909091,34,m,straight,athletic,strictly anything,socially,never,has a kid,likes dogs and likes cats,judaism and laughing about it,no
4532,0.909091,34,m,straight,fit,anything,socially,never,has a kid,likes dogs and likes cats,agnosticism and laughing about it,no
43965,0.909091,34,m,straight,athletic,mostly anything,rarely,never,has a kid,has dogs and has cats,catholicism but not too serious about it,no
41942,0.909091,34,m,straight,fit,anything,not at all,never,has a kid,likes dogs and likes cats,catholicism but not too serious about it,no
56272,0.909091,38,m,straight,fit,mostly anything,often,never,has kids,likes dogs and has cats,atheism,no


#### **--> Analysis: Straight Female**

**Initial Matches based on matching criteria + cosine similarity**
- Based on the matching criteria, the user input is that of a **straight female** with an **age range between 32 - 40**, who **doesn't have kid(s), but wants kid(s)**, and religion **doesn't matter** for them. A user who's religion doesn't matter returns other users who are atheist or not serious/laughing about their religion. The initial recommendations returned straight males who all have kids.
- In terms of the other features that weren't part of the matching criteria, the user input has a diet consisting of `anything`, `yes` to drinks, and `no` to drugs, all returning recommended profiles similar for the most part, aside from user `3497` who indicated that they don't drink. Additionally, the user has body type of `full figured`, with only profile `3497` as a `curvy` body type.

**Liked Profiles + Additional Matches**
- Although profile `3497` is the most simlar in terms of body type, relative to drinking habits, the User liked other profiles. For example, profile `7206` was a liked profile, since the straight male is of a `fit` body type, a religion that they're laughing about, and likes dogs and has cats, despite the User having a `full figured` body type. Additionally, the User liked profile `15047`, to like an additional profile with a `fit` body type. Similarly, this straight male profile has kids and likes dogs, similar to that of the User's preferences.
- In terms of the **additional recommended profiles**, they are based off the liked profiles the User specified, while still performing cosine similarity against a filtered dataset based on the User's age range, orientation, and religion preference. The only matching criteria that is *directly related* to liked profile is `offspring sentiment`. Since both liked profiles are of `fit` body type, the additional matches returned have users that are either `fit` or `athletic`.
- Additionally, despite both profiles having drinking habits of `often`, one the the additional matches `4192` didn't drink at all. However, relative to the other features, the additional matches are relatively similar to the User's original and liked preferences.

In [266]:
# bisexual male

invalue12 = ['43', 'm', 'bisexual', 'thin', 'anything', 'yes', 'no', "doesn't have kid(s), but wants kid(s)", 'dislikes dogs and cats', "doesn't matter", 'yes']
highest_age12 = 45
lowest_age12 = 38
religion12 = "doesn't matter"

lover_recommender_test8(invalue12, religion12, lowest_age12, highest_age12)

C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,similarity,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
user_id,,,,,,,,,,,,
21136,0.545455,44,m,bisexual,athletic,mostly anything,often,sometimes,"has kids, but doesn't want more",dislikes dogs and dislikes cats,atheism but not too serious about it,no
48663,0.545455,45,m,bisexual,fit,mostly anything,socially,never,"has a kid, and might want more",dislikes dogs and dislikes cats,agnosticism but not too serious about it,when drinking
13793,0.545455,45,m,bisexual,jacked,mostly anything,not at all,never,has kids,dislikes dogs and dislikes cats,other and laughing about it,no
15838,0.545455,42,m,bisexual,average,anything,socially,never,has a kid,dislikes dogs and dislikes cats,atheism,no
43765,0.545455,38,m,bisexual,a little extra,anything,socially,never,has kids,dislikes dogs and dislikes cats,agnosticism and laughing about it,no


In [316]:
# returning additional profiles based on any of the ones the user liked

answer = str(input('Any profile(s) catch your eye? (yes/no)'))

profile_lis = []
while answer == 'yes':
    profile = int(input('Name one profile (user_id) of interest: '))
    profile_lis.append(profile)
    answer = str(input('Any other profile(s) caught your eye? (yes/no)'))
    
more_matches2(invalue12, profile_lis, religion12, lowest_age12, highest_age12)

Any profile(s) catch your eye? (yes/no) yes
Name one profile (user_id) of interest:  48663
Any other profile(s) caught your eye? (yes/no) no
Want to see more profiles? (yes/no) yes


Here are more profiles, based on the profile(s) of interest from earlier:


C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,similarity,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
user_id,,,,,,,,,,,,
314,0.727273,45,m,gay,athletic,anything,socially,never,has kids,dislikes dogs and dislikes cats,judaism but not too serious about it,no
13793,0.727273,45,m,bisexual,jacked,mostly anything,not at all,never,has kids,dislikes dogs and dislikes cats,other and laughing about it,no
46640,0.636364,42,f,bisexual,athletic,anything,socially,never,has kids,dislikes dogs and dislikes cats,atheism,no
33255,0.636364,39,f,straight,fit,anything,socially,never,has a kid,dislikes dogs and dislikes cats,atheism,sometimes
5062,0.636364,40,f,bisexual,athletic,anything,socially,never,has kids,dislikes dogs and dislikes cats,atheism and laughing about it,no


#### **--> Analysis: Bisexual Male**

**Initial Matches based on matching criteria + cosine similarity**
- Based on the matching criteria, the user input is that of a **bisexual male** with an **age range between 38 - 45**, who **doesn't have kid(s), but wants kid(s)**, and religion **doesn't matter** for them. A user who's religion doesn't matter returns other users who are atheist or not serious/laughing about their religion. The initial recommendations returned all bisexual males who all have kids as well.
- In terms of the other features that weren't part of the matching criteria, the user input has a diet consisting of `anything`, `yes` to drinks, and `no` to drugs, all returning recommended profiles similar for the most part, as well as all profiles who `dislikes dogs and cats`, aside from user `21136` who indicated that they sometimes takes drugs. Additionally, the user indicated that they smoke, whereas only profile `48663` indicated that they smoke, regardless if it's `sometimes`.

**Liked Profiles + Additional Matches**
- Since the User smokes, they liked profile `48663`, where the profile smokes sometimes. Based on the liked profile, the additional matches include a gay male, bisexual male and females, and a straight female. 
- In terms of the **additional recommended profiles**, they are based off the liked profiles the User specified, while still performing cosine similarity against a filtered dataset based on the User's age range, orientation, and religion preference. The only matching criteria that is *directly related* to liked profile is `offspring sentiment`, which is recommending more profiles that have kids. Since liked profile `48663` had a body type of `fit`, the additional profiles are similarly `athletic`, `jacked`, and `fit`.
- Again, only one additional match `33255` indicated that a user smoked, despite it being `sometimes`.

In [264]:
# bisexual female

invalue5 = ['26', 'f', 'bisexual', 'average', 'vegan', 'no', 'no', "doesn't have kids, and doesn't want any", 'likes cats', "buddhism", 'yes']
highest_age5 = 30
lowest_age5 = 22
religion5 = "buddhism"

lover_recommender_test8(invalue5, religion5, lowest_age5, highest_age5)

C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,similarity,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
user_id,,,,,,,,,,,,
47399,0.454545,27,m,straight,average,strictly anything,not at all,never,doesn't have kids,likes dogs,buddhism,yes
48349,0.454545,30,m,straight,fit,vegan,not at all,never,"doesn't have kids, and doesn't want any",likes dogs and has cats,buddhism,no
26260,0.454545,22,f,bisexual,skinny,strictly vegan,socially,never,doesn't have kids,likes dogs and has cats,buddhism,trying to quit
24689,0.454545,29,m,straight,average,other,not at all,never,doesn't have kids,has cats,buddhism,no
31845,0.454545,27,f,bisexual,curvy,mostly vegetarian,not at all,never,doesn't want kids,likes dogs and likes cats,buddhism and somewhat serious about it,no


In [317]:
# returning additional profiles based on any of the ones the user liked

answer = str(input('Any profile(s) catch your eye? (yes/no)'))

profile_lis = []
while answer == 'yes':
    profile = int(input('Name one profile (user_id) of interest: '))
    profile_lis.append(profile)
    answer = str(input('Any other profile(s) caught your eye? (yes/no)'))
    
more_matches2(invalue5, profile_lis, religion5, lowest_age5, highest_age5)

Any profile(s) catch your eye? (yes/no) yes
Name one profile (user_id) of interest:  48349
Any other profile(s) caught your eye? (yes/no) yes
Name one profile (user_id) of interest:  24689
Any other profile(s) caught your eye? (yes/no) no
Want to see more profiles? (yes/no) yes


Here are more profiles, based on the profile(s) of interest from earlier:


C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,similarity,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
user_id,,,,,,,,,,,,
19471,0.727273,26,m,straight,athletic,vegan,not at all,never,doesn't have kids,dislikes dogs and dislikes cats,buddhism,no
29713,0.727273,30,m,straight,athletic,anything,socially,never,doesn't have kids,has dogs and likes cats,buddhism,no
4813,0.727273,30,m,straight,athletic,vegetarian,not at all,never,doesn't have kids,likes dogs and likes cats,buddhism and somewhat serious about it,sometimes
11348,0.727273,30,m,straight,fit,anything,socially,never,doesn't have kids,likes dogs and likes cats,buddhism,no
50294,0.727273,30,m,straight,fit,anything,not at all,never,doesn't have kids,likes dogs,buddhism and somewhat serious about it,no


#### **--> Analysis: Bisexual Female**

**Initial Matches based on matching criteria + cosine similarity**
- Based on the matching criteria, the user input is that of a **straight female** with an **age range between 22 - 30**, who **doesn't have kids, and doesn't want any**, and **buddhism** religion.  A user specifying their religion indicates that religion matters to them, so the recommendations return other users who are Buddhist / serious about Buddhism as well. The initial recommendations returned a mix of straight males and bisexual females who all don't have kids.
- In terms of the other features that weren't part of the matching criteria, the user input has a diet consisting of `vegan`, `doesn't drink at all`, and `no` to drugs. Only profile `26260` indicated that they drank `socially`. Additionally, those of `vegan`, `vegetarian`, and `anything` diets were also recommended. In terms of pet sentiment, the User noted that she liked cats only, however profile `47399` indicated that the profile liked dogs only.

**Liked Profiles + Additional Matches**
- Although the User indicated that she smoked, the profiles she liked were `48349` and `24689`, since the pet sentiments most closely represents that of the user -- both profiles indicated that they have cats, whereas the User likes cats.
- In terms of the **additional recommended profiles**, they are based off the liked profiles the User specified, while still performing cosine similarity against a filtered dataset based on the User's age range, orientation, and religion preference. The only matching criteria that is *directly related* to liked profile is `offspring sentiment`. Since both liked profiles were straight males who didn't smoke, with a `fit` and `average` body type, all additional matches are straight males that have a `fit` or `athletic` body type, with all but one profile indicating that they didn't smoke.
- Additionally, because liked profile `48349` indicated that they like dogs and cats, the addiitonal matches have a mix of users who like both pets, dislike both pets, or just like dogs -- this feature doesn't match the pet sentiment of the user, but instead *moreso the liked profiles*.

In [280]:
# gay male

invalue9 = ['25', 'm', 'gay', 'average', 'anything', 'yes', 'no', "has kid(s), but doesn't want more", 'likes dogs and cats', "doesn't matter", 'no']
highest_age9 = 35
lowest_age9 = 28
religion9 = "doesn't matter"

lover_recommender_test8(invalue9, religion9, lowest_age9, highest_age9)

C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,similarity,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
user_id,,,,,,,,,,,,
16775,0.818182,29,m,gay,average,mostly anything,often,never,doesn't have kids,likes dogs and likes cats,atheism,no
46102,0.818182,28,m,gay,average,strictly anything,often,never,doesn't have kids,likes dogs and likes cats,judaism but not too serious about it,no
45626,0.818182,30,m,gay,average,anything,often,never,doesn't have kids,likes dogs and has cats,christianity but not too serious about it,no
42964,0.727273,29,m,gay,average,strictly anything,not at all,never,doesn't have kids,likes dogs and likes cats,atheism,no
35743,0.727273,28,m,gay,average,strictly anything,often,never,doesn't have kids,dislikes dogs and dislikes cats,atheism,no


In [318]:
# returning additional profiles based on any of the ones the user liked

answer = str(input('Any profile(s) catch your eye? (yes/no)'))

profile_lis = []
while answer == 'yes':
    profile = int(input('Name one profile (user_id) of interest: '))
    profile_lis.append(profile)
    answer = str(input('Any other profile(s) caught your eye? (yes/no)'))

more_matches2(invalue9, profile_lis, religion9, lowest_age9, highest_age9)

Any profile(s) catch your eye? (yes/no) yes
Name one profile (user_id) of interest:  16775
Any other profile(s) caught your eye? (yes/no) yes
Name one profile (user_id) of interest:  46102
Any other profile(s) caught your eye? (yes/no) yes
Name one profile (user_id) of interest:  45626
Any other profile(s) caught your eye? (yes/no) no
Want to see more profiles? (yes/no) yes


Here are more profiles, based on the profile(s) of interest from earlier:


C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,similarity,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
user_id,,,,,,,,,,,,
42650,0.909091,29,m,gay,average,anything,socially,never,doesn't have kids,likes dogs and likes cats,atheism,no
15352,0.909091,29,m,gay,fit,anything,often,never,doesn't have kids,likes dogs and likes cats,atheism and very serious about it,no
48214,0.909091,29,m,gay,average,strictly anything,often,sometimes,doesn't have kids,likes dogs and likes cats,atheism but not too serious about it,no
22893,0.909091,29,m,gay,average,mostly anything,socially,never,doesn't have kids,likes dogs and has cats,atheism,no
46012,0.909091,29,m,gay,average,mostly anything,not at all,never,doesn't have kids,likes dogs and likes cats,agnosticism but not too serious about it,no


#### **--> Analysis: Gay Male**

**Initial Matches based on matching criteria + cosine similarity**
- Based on the matching criteria, the user input is that of a **gay male** with an **age range between 28 - 35**, who **has kid(s), but doesn't want more**, and religion **doesn't matter** for them. A user who's religion doesn't matter returns other users who are atheist or not serious/laughing about their religion. The initial recommendations returned gay males who all don't have kids.
- In terms of the other features that weren't part of the matching criteria, the user input has a diet consisting of `anything`, `yes` to drinks, and `no` to drugs, returning recommended profiles similar for the most part, aside from profile `42964` who indicated that they don't drink and profile `35743` who indicated that they dislike dogs and cats.

**Liked Profiles + Additional Matches**
- Profiles `16775`, `46102`, and `45626` were liked by the User, since across the board, the features were similar to the User's inputs.
- In terms of the **additional recommended profiles**, they are based off the liked profiles the User specified, while still performing cosine similarity against a filtered dataset based on the User's age range, orientation, and religion preference. The only matching criteria that is *directly related* to liked profile is `offspring sentiment`. Since all liked profiles are gay males who don't have kids, all additional profiles were also gay males who don't have kids.
- Additionally, there are still 2 additional matches that are slightly different from the User's profile/lifestyle -- profile `48214` that takes drugs `sometimes` and profile `46012` who doesn't drink, whereas the User indicated that they drink and don't take drugs.

In [152]:
# gay female

invalue7 = ['30', 'f', 'gay', 'rather not say', 'vegetarian', 'yes', 'yes', "doesn't have kid(s), but wants kid(s)", 'dislikes dogs and cats', "christianity", 'yes']
highest_age7 = 45
lowest_age7 = 32
religion7 = "christianity"

lover_recommender_test8(invalue7, religion7, lowest_age7, highest_age7)

C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,similarity,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
user_id,,,,,,,,,,,,
25664,0.363636,35,f,bisexual,curvy,mostly anything,often,never,"has kids, and might want more",has dogs,christianity,yes
49211,0.363636,32,f,gay,average,anything,socially,never,has a kid,dislikes dogs and dislikes cats,christianity,no
24990,0.272727,32,f,bisexual,average,anything,rarely,never,has kids,dislikes dogs and dislikes cats,christianity and somewhat serious about it,sometimes
42459,0.181818,43,f,bisexual,athletic,mostly anything,socially,never,"has kids, but doesn't want more",likes dogs and has cats,christianity and somewhat serious about it,no


In [310]:
# returning additional profiles based on any of the ones the user liked

answer = str(input('Any profile(s) catch your eye? (yes/no)'))

profile_lis = []
while answer == 'yes':
    profile = int(input('Name one profile (user_id) of interest: '))
    profile_lis.append(profile)
    answer = str(input('Any other profile(s) caught your eye? (yes/no)'))

more_matches2(invalue7, profile_lis, religion7, lowest_age7, highest_age7)

Any profile(s) catch your eye? (yes/no) yes
Name one profile (user_id) of interest:  49211
Any other profile(s) caught your eye? (yes/no) yes
Name one profile (user_id) of interest:  24990
Any other profile(s) caught your eye? (yes/no) no
Want to see more profiles? (yes/no) yes


Here are more profiles, based on the profile(s) of interest from earlier:


C:\Users\veron\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,similarity,age,sex,orientation,body_type,diet,drinks,drugs,offspring,pets,religion,smokes
user_id,,,,,,,,,,,,
58513,0.818182,42,f,gay,a little extra,mostly anything,socially,never,doesn't have kids,dislikes dogs and dislikes cats,christianity,no
42052,0.727273,33,f,gay,average,anything,socially,never,doesn't want kids,has dogs,christianity,no
52501,0.727273,34,f,gay,average,anything,not at all,never,"doesn't have kids, but wants them",dislikes dogs and dislikes cats,christianity,no
6254,0.636364,44,f,gay,curvy,strictly other,socially,never,doesn't want kids,dislikes dogs and dislikes cats,christianity and very serious about it,no
47403,0.636364,44,f,gay,average,anything,rarely,never,doesn't have kids,likes dogs and has cats,christianity and somewhat serious about it,sometimes


#### **--> Analysis: Gay Female**

**Initial Matches based on matching criteria + cosine similarity**
- Based on the matching criteria, the user input is that of a **gay female** with an **age range between 32 - 45**, who **doesn't have kid(s), but wants kid(s)**, and **Christian** for them.  A user specifying their religion indicates that religion matters to them, so the recommendations return other users who are Christian / serious about Christianity as well. The initial recommendations returned a mix of gay and bisexual females who have kids.
- In terms of the other features that weren't part of the matching criteria, since the user input has body type `rather not say`, the initial matches have a mix of `average`, `curvy`, and `athletic` body types. Additionally, only four matches returned, potentially due to the combination of user inputs and matching criteria that had a small pool of users to recommend. After all, there are only 1.6k Christianity profiles overall, not including filtering based on additional matching criteria. similarly, The user inputted `yes` to drugs, whereas all initial matches had users who don't take drugs. Similarly, there are just under 400 profiles overall of users who noted that they take drugs.

**Liked Profiles + Additional Matches**
- Since the User input indicated that they dislike dogs and cats, the only liked profiles are `49211` and `24990`. Although the liked profiles only drink sometimes (`socially`, `rarely`), the drinking habits are still somewhat related to the User, as well as profile `24990` indicating that they smoked sometimes, matching the User's lifestyle of smoking.
- In terms of the **additional recommended profiles**, they are based off the liked profiles the User specified, while still performing cosine similarity against a filtered dataset based on the User's age range, orientation, and religion preference. The only matching criteria that is *directly related* to liked profile is `offspring sentiment`. Interestingly, the additional matches returned a mix of profiles who don't have kids or doesn't want kids, whereas the User's offspring sentiment was that they `don't have kid(s), but wants kid(s)`
- Despite one of the liked profiles `24990` being that of a bisexual female, the additional matches were all gay females. Additionally, since both liked profiles drank sometimes (`socially`, `rarely`) and both `disliked dogs and cats`, 3 out of 5 of the matches were those who drank socially and disliked dogs and cats